In [135]:
import scipy.stats as sts
import numpy as np
import pandas as pd
np.random.seed(0)

# Estimators
def nonParametricOptimalQuantity(distributionData, tau):
    optimalQuantity = np.quantile(distributionData,tau)
    return optimalQuantity

def profit(tau, demand, quantity):
    price = 1
    cost = 1 - tau
    return price * np.minimum(demand,quantity) - cost * quantity

def parametricOptimalQuantity(estimators, tau):
    optimalQuantity = sts.norm.ppf(tau, loc = estimators[0], scale = estimators[1])
    return optimalQuantity

# Evaluation of test statistics functions
def empericalRootMeanSquaredError(m, optimalQuantities, tau, meanDemand, stdevDemand):
    realOptimalQuantity = parametricOptimalQuantity([meanDemand, stdevDemand], tau)
    rmse = np.sqrt(1/m * np.sum((optimalQuantities - realOptimalQuantity)**2))
    return rmse

def empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand):
    return empericalRootMeanSquaredError(m, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand) / empericalRootMeanSquaredError(m, parametricOptimalQuantitys, tau, meanDemand, stdevDemand)

def empericalServiceLevel(m, optimalQuantities, demand):
    indicatorFunction = np.where(optimalQuantities.T >= np.array(demand), 1, 0)
    return 1 / m * np.sum(indicatorFunction)

def empericalProfitLoss(m, optimalQuantities, demand, tau, meanDemand, stdevDemand):
    realOptimalQuantity = parametricOptimalQuantity([meanDemand, stdevDemand], tau)
    profitReal = profit(tau, meanDemand, realOptimalQuantity)
    profitEstimator = profit(tau, demand, optimalQuantities.T)
    return 1 / m * np.sum(np.abs((profitReal - profitEstimator) /  profitReal))

def empericalProfitLossRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand):
    return empericalProfitLoss(m, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand) / empericalProfitLoss(m, parametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand)

def monteCarlo(m, tau, n, meanDemand, stdevDemand):
    # Define arrays
    parametricOptimalQuantitys = np.empty((m,1))
    nonParametricOptimalQuantitys = np.empty((m,1))
    allParameters = np.empty((m,2))

    # Compute estimators and optimal quantities
    for j in range(m):
        distributionData = sts.norm.rvs(loc = meanDemand, scale = stdevDemand, size = n)
        allParameters[j] = parameters = sts.norm.fit(distributionData)
        parametricOptimalQuantitys[j] = parametricOptimalQuantity(parameters, tau)
        nonParametricOptimalQuantitys[j] = nonParametricOptimalQuantity(distributionData, tau)
    
    demand = allParameters[:,0] # in case of normal

    # Compute evaluation statistics
    rmse = empericalRootMeanSquaredErrorRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, tau, meanDemand, stdevDemand)
    eslParametric = empericalServiceLevel(m, parametricOptimalQuantitys, demand)
    eslNonParametric = empericalServiceLevel(m, nonParametricOptimalQuantitys, demand)
    eplr = empericalProfitLossRatio(m, parametricOptimalQuantitys, nonParametricOptimalQuantitys, demand, tau, meanDemand, stdevDemand)
    result = {
                    'MonteCarlo iterations' : m,
                    'Sample Size': n,
                    'Target Service Level': tau,
                    'Param Values': np.mean(allParameters, axis=0),
                    'RMSE Ratio': rmse,
                    'SL nonParam': eslParametric,
                    'SL Param': eslNonParametric,
                    'EPLR': eplr
                }
    return result





In [136]:
numberOfMontecarloIterations = 1000
stdevDemand = 15
meanDemand = 120

tauArray = [0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
nArray = [10, 50, 100, 200]

results = []
for tau in tauArray:
    for n in nArray:
        result = monteCarlo(numberOfMontecarloIterations, tau, n, meanDemand, stdevDemand)
        results.append(result)


In [137]:
df = pd.DataFrame(results)
df

,MonteCarlo iterations,Sample Size,Target Service Level,Param Values,RMSE Ratio,SL nonParam,SL Param,EPLR
0,1000,10,0.01,"[119.72349419762591, 13.720362743553597]",1.644325,0.0,0.000,1.771051
1,1000,50,0.01,"[120.0140967807625, 14.740757309842326]",1.640041,0.0,0.000,1.683588
2,1000,100,0.01,"[120.14130607280387, 14.858540982582157]",1.871786,0.0,0.000,1.941332
3,1000,200,0.01,"[119.98878184228278, 14.947526742899928]",1.903595,0.0,0.000,1.944169
4,1000,10,0.05,"[120.11252657015035, 13.742808256866427]",1.207038,0.0,0.000,1.243371
5,1000,50,0.05,"[120.13138641676703, 14.761916927166336]",1.303617,0.0,0.000,1.301021
6,1000,100,0.05,"[120.02817687096912, 14.90756341535131]",1.299937,0.0,0.000,1.305279
7,1000,200,0.05,"[120.0361744493198, 14.940970788928059]",1.360500,0.0,0.000,1.377528
8,1000,10,0.10,"[119.83957962745934, 13.861372332356554]",1.184618,0.0,0.000,1.204583
9,1000,50,0.10,"[119.98330792635866, 14.757411232445705]",1.251779,0.0,0.000,1.261884


In [138]:
import plotly.express as px

df_f = df[df["Sample Size"]==200]

fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, higher RMSE means parametric performs better")
fig.show()

fig = px.scatter(df, x='Target Service Level', y='EPLR', color="Sample Size", title="TSL vs EPLR, higher EPLR means non parametric has a higher profit loss")
fig.show()

#fig = px.scatter(df, x='Target Service Level', y='RMSE Ratio', color="Sample Size", title="TSL vs RMSE, higher RMSE means parametric performs better")
#fig.show()
